# EJERCICIO REGLAS DE ASOCIACION

In [ ]:
library(readxl)
library(arules)
library(dplyr)

In [ ]:
df <- read_excel("../input/dataset-ra/EjercicioReglasAsociacion.xlsx")

**a)** ¿Cuántas transacciones tiene la base? 

Primero calculamos la cantidad de filas del dataset y obtenemos que son 596.

In [ ]:
nrow(df)

**b)** ¿Cuál es la transacción “más larga”?

Primero transformamos nuestro dataset en una matrix de 0 y 1 y calculamos la maxima columna segun el largo de los items.

In [ ]:
### remove transaction IDs
tid <- as.character(df[["Nro Transaccion"]])
df <- df[,-1]

# Limpiamos el dataset y cada X pasa a tener el valor de su columna y cada '-' pasa a ser NA
df <- df %>% 
    transmute_all(funs(ifelse(. == "X", 1, 0)))

In [ ]:
dfAux <- cbind(df, Length = rowSums(df))
max <- dfAux[which.max(dfAux$Length),]
max

*c)* ¿Cuál es el 1-itemset más frecuente? 

Para esto calculamos la suma de las columnas y buscamos el maximo. De esta forma encontramos que es M con 46 apariciones. 

In [ ]:
sort(colSums(df), decreasing = TRUE)

**d)** ¿Cuántas transacciones de 3 ítems hay? 

Contamos todos los itemsets con longitus 3 y obtenemos que son 46.

In [ ]:
sum(dfAux$Length == 3)

**e)** Tome las 3 primeras letras de su nombre, calcule el soporte para cada uno de los 1-itemset, 2-itemsets y 3-itemsets que se pueden formar con la misma. Enuncie todas las posibles reglas que se pueden construir y calcule el soporte, la confianza y el lift para cada una.

Estas serian: 

Soporte 1-itemset: {D} = 23/596  
Soporte 1-itemset: {A} = 18/596  
Soporte 1-itemset: {V} = 53/596  

In [ ]:
dav <- df[c("D","A","V")]
sort(colSums(dav), decreasing = TRUE)

Soporte 2-itemset: {D,A} = 1/596  
Soporte 2-itemset: {A,V} = 0/596  
Soporte 2-itemset: {D,V} = 0/596  

In [ ]:
da <- df[c("D","A")]
da <- cbind(da, Length = rowSums(da))
sum(da$Length == 2)

av <- df[c("A","V")]
av <- cbind(av, Length = rowSums(av))
sum(av$Length == 2)

dv <- df[c("D","V")]
dv <- cbind(dv, Length = rowSums(dv))
sum(dv$Length == 2)

Soporte 3-itemset: {D,A,V} = 0/596

In [ ]:
dav <- df[c("D","A","V")]
dav <- cbind(da, Length = rowSums(dav))
sum(dav$Length == 3)



|**Reglas posibles:** |      |            |             |           |     |
| ---          |             |            |             |           |     |
| lhs   &  rhs | support     | confidence | coverage    | lift      |count|
| {A}   => {F} | 0.0067114   | 0.22222    | 0.0302013   |  5.5185   |4    |
| {A}   => {N} | 0.0067114   | 0.22222    | 0.0302013   |  5.5185   |4    |
| {A}   => {O} | 0.0050335   | 0.16666    | 0.0302013   |  4.1388   |3    |
| {A}   => {C} | 0.0083892   | 0.27777    | 0.0302013   |  4.1388   |5    |
| {A}   => {L} | 0.0050335   | 0.16666    | 0.0302013   |  1.7426   |3    |
| {A}   => {H} | 0.0050335   | 0.16666    | 0.0302013   |  1.6021   |3    |
| {A,F} => {C} | 0.0067114   | 1.00000    | 0.0067114   | 14.9000   |4    |
| {A,C} => {F} | 0.0067114   | 0.80000    | 0.0083892   | 19.8666   |4    |


**f)** ¿Cuál debería ser el min support y el min confidence para que las reglas enunciadas en el
ejercicio anterior formarán parte del resultado de la aplicación del algoritmo “a priori”?


En esta parte calculamos apriori para ver que reglas se nos generan. Para utilizar apriori convertirmos el dataset a una lista de transacciones.

In [ ]:
obtenerReglas <- function(df,minsupport=0.05,confianza=0.9){
    reglas <- apriori(df, parameter=list(support=minsupport,confidence = confianza, minlen=2 ))
    #Nos aseguramos de que tengan mas de una aparicion
    reglas<- subset(reglas, subset = count>2)
    inspect(sort(reglas,by=c('lift','confidence')))
    return (reglas)
}

In [ ]:
Matrix1 = data.matrix(df)
trans <- as(Matrix1, "transactions")

In [ ]:
inspect(trans)

In [ ]:
summary(trans)

In [ ]:
reglas <- obtenerReglas(trans,0.005,0.15) # El support elegido fue obtenido de forma empirica
                                        # Primero probe con 0.001 y fui incrementando hasta encontrar el minimo para las obtener las reglas mencionadas

In [ ]:
#Buscamos el subset de reglas de interes 
rules.sub <- subset(reglas, subset = lhs %in% c("D","A","V"))
inspect(rules.sub)